# Final

### Computer Vision 

In [ ]:
import os
import cv2
import numpy as np
from skimage.filters import frangi
import matplotlib.pyplot as plt
from glob import glob

def preprocess_leaf_image(image_path, output_size=(224, 224)):
    """
    Preprocess a leaf image to extract RGB, venation map, and edge map
    
    Args:
        image_path (str): Path to the leaf image
        output_size (tuple): Size to resize images to (height, width)
        
    Returns:
        tuple: (RGB image, venation map, edge map)
    """
    # Read the original image
    original_image = cv2.imread(image_path)
    if original_image is None:
        raise ValueError(f"Could not read image at {image_path}")
    
    # 1. RGB Image Extraction
    rgb_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    rgb_image = cv2.resize(rgb_image, output_size)
    
    # 2. Venation Map Extraction
    # Convert to grayscale
    gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    gray_image = cv2.resize(gray_image, output_size)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_image = clahe.apply(gray_image)
    
    # Apply Frangi filter to enhance vein-like structures
    # Updated parameters to be compatible with current scikit-image version
    venation_map = frangi(clahe_image, 
                         scale_range=(1, 3), 
                         scale_step=0.5,
                         beta=15,  # Using beta instead of beta1/beta2
                         black_ridges=False)
    
    # Normalize the venation map
    venation_map = cv2.normalize(venation_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    # 3. Edge Map Extraction using Canny edge detection
    edge_map = cv2.Canny(gray_image, 50, 150)
    
    return rgb_image, venation_map, edge_map

def process_dataset(dataset_path, output_dir):
    """
    Process an entire dataset of leaf images
    
    Args:
        dataset_path (str): Path to the dataset directory
        output_dir (str): Directory to save processed images
    """
    # Create output directories if they don't exist
    os.makedirs(os.path.join(output_dir, 'rgb'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'venation'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'edge'), exist_ok=True)
    
    # Get all image files
    image_extensions = ['*.jpg', '*.jpeg', '*.png']
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob(os.path.join(dataset_path, '**', ext), recursive=True))
    
    print(f"Found {len(image_files)} images to process")
    
    for idx, image_path in enumerate(image_files):
        try:
            # Extract class from path (assuming dataset structure: dataset/class/image.jpg)
            # Handle paths with spaces correctly
            path_parts = os.path.normpath(image_path).split(os.sep)
            class_name = path_parts[-2]  # Assuming last directory is the class name
            file_name = os.path.basename(image_path)
            
            # Create class directories if they don't exist
            os.makedirs(os.path.join(output_dir, 'rgb', class_name), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'venation', class_name), exist_ok=True)
            os.makedirs(os.path.join(output_dir, 'edge', class_name), exist_ok=True)
            
            # Process the image
            rgb_image, venation_map, edge_map = preprocess_leaf_image(image_path)
            
            # Save processed images
            cv2.imwrite(os.path.join(output_dir, 'rgb', class_name, file_name), 
                        cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))
            cv2.imwrite(os.path.join(output_dir, 'venation', class_name, file_name), venation_map)
            cv2.imwrite(os.path.join(output_dir, 'edge', class_name, file_name), edge_map)
            
            if idx % 100 == 0:
                print(f"Processed {idx}/{len(image_files)} images")
                
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

def visualize_preprocessing(image_path):
    """
    Visualize the preprocessing steps for a single image
    
    Args:
        image_path (str): Path to the leaf image
    """
    rgb_image, venation_map, edge_map = preprocess_leaf_image(image_path)
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(rgb_image)
    plt.title('RGB Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(venation_map, cmap='gray')
    plt.title('Venation Map')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(edge_map, cmap='gray')
    plt.title('Edge Map')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Function to check scikit-image version and available parameters
def check_frangi_parameters():
    """
    Print the available parameters for the frangi function
    """
    import inspect
    from skimage import __version__ as skimage_version
    
    print(f"scikit-image version: {skimage_version}")
    print("Frangi function parameters:")
    print(inspect.signature(frangi))
    print("For more details, refer to the documentation at:")
    print("https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.frangi")

# Example usage
if __name__ == "__main__":
    # Check frangi parameters to diagnose issues
    check_frangi_parameters()
    
    # Path to the Mendeley leaf dataset
    dataset_path = "/Users/arnavkarnik/Documents/Recognition-of-Medicinal-Plant-Species-Deep-Learning-Project--Sem6/Medicinal Leaf Dataset/Segmented Medicinal Leaf Images"

    
    # Process the entire dataset
    process_dataset(dataset_path, output_dir)
    
    # Alternatively, visualize preprocessing for a single image
    # sample_image = "/Users/arnavkarnik/Documents/Recognition-of-Medicinal-Plant-Species-Deep-Learning-Project--Sem6/Medicinal Leaf Dataset/Segmented Medicinal Leaf Images/Punica Granatum (Pomegranate)/PG-S-022.jpg"
    # visualize_preprocessing(sample_image)

scikit-image version: 0.24.0
Frangi function parameters:
(image, sigmas=range(1, 10, 2), scale_range=None, scale_step=None, alpha=0.5, beta=0.5, gamma=None, black_ridges=True, mode='reflect', cval=0)
For more details, refer to the documentation at:
https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.frangi
Found 1835 images to process
Processed 0/1835 images


/var/folders/0b/d5vghlcn39jb_kt0c4whkzcr0000gn/T/ipykernel_7610/2342296929.py:39: UserWarning: Use keyword parameter `sigmas` instead of `scale_range` and `scale_range` which will be removed in version 0.17.
  venation_map = frangi(clahe_image,


Processed 100/1835 images
Processed 200/1835 images
Processed 300/1835 images
Processed 400/1835 images
Processed 500/1835 images
Processed 600/1835 images
Processed 700/1835 images
Processed 800/1835 images
Processed 900/1835 images
Processed 1000/1835 images
Processed 1100/1835 images
Processed 1200/1835 images
Processed 1300/1835 images
Processed 1400/1835 images
Processed 1500/1835 images
Processed 1600/1835 images
Processed 1700/1835 images
Processed 1800/1835 images


# Model 1

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Define paths
data_dir = "/Users/arnavkarnik/Documents/Recognition-of-Medicinal-Plant-Species-Deep-Learning-Project--Sem6/Output/"
train_dir = data_dir + "train"
val_dir = data_dir + "val"

# Image Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Load ResNet50 Base Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model initially

# Add Custom Layers
x = Flatten()(base_model.output)
x = Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)
output_layer = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for Improvement
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[reduce_lr, early_stopping],
    verbose=1)

# Evaluate Model
y_true = np.concatenate([val_generator.classes])
y_pred = np.argmax(model.predict(val_generator), axis=1)
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys()))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


FileNotFoundError: [Errno 2] No such file or directory: '/Users/arnavkarnik/Documents/Recognition-of-Medicinal-Plant-Species-Deep-Learning-Project--Sem6/Output/train'